# Learning RL using StableBaseline3

### Importing Necessary libraries

In [20]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

Here We are using the inbuild example environment of Cartpole environemnt


In [23]:
#Making the vatiable for the model name
env_name = "CartPole-v1"
#Creating the environment , remnder_mode is set to human for visualisation
env = gym.make(env_name,  render_mode="human")

Understanding the environment class menthods

In [19]:
#Checking what are the outputs from the resetting the environment
print(env.reset())
# Checking the action and observation space of the environment
print(env.action_space)
print(env.observation_space)
#Observing the output of the step function with the action 1
obs, reward, done, info, _ = env.step(1)
print("obs : ", obs,"reward : ", reward,"Done : " ,done,"info : " ,info)
# closing the environment to close the Visualisation window
env.close()

(array([ 0.04225325, -0.03066197, -0.03478266,  0.04284509], dtype=float32), {})
Discrete(2)
Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
obs :  [ 0.04164001  0.16494104 -0.03392576 -0.26060602] reward :  1.0 Done :  False info :  False


Now Lets run the episode of balacing the cartpole 
and assuming we are not learning or trying to learn to balance

In [ ]:
# Lets define the number of episodes that is
# we will run the simulation for number of episodes time just 
# to see how the environment is working and how the action space is being used
# and how the reward is being calculated
episodes = 10 # Defineing the number of episodes
for episode in range(1, episodes + 1): #for loop to run the simulation for number of episodes
    # Reset the environment for each episode
    obs = env.reset()
    # Initialize done variable to False which is used to raise flage of termination
    done = False
    # Variable to keep track of the score; Score is cumulative reward
    score=0
    # Loop until the episode is done (or not terminated)
    #we could have also used max reward or max time steps to terminate the episode
    while not done:
        # Sample a random action from the action space
        # In a real scenario, you would use a trained model to predict the action
        action= env.action_space.sample()  # Random action for demonstration
        # Take a step in the environment with the sampled action
        # The step function returns the next observation, reward, done flag, info dictionary and the time step
        obs, reward, done, info, _ = env.step(action)
        # render the environment to visualize the action taken
        env.render()
        #keep the score of the episode
        score += reward
        # print(f"Episode: {episode}, Score: {score}, Action: {action}, Reward: {reward}")
    print(f"Episode {episode} finished with score: {score}")

env.close()  # Close the environment after all episodes are done
    

Episode 1 finished with score: 16.0
Episode 2 finished with score: 19.0
Episode 3 finished with score: 23.0
Episode 4 finished with score: 38.0
Episode 5 finished with score: 20.0
Episode 6 finished with score: 17.0
Episode 7 finished with score: 29.0
Episode 8 finished with score: 26.0
Episode 9 finished with score: 53.0
Episode 10 finished with score: 21.0


So in above logs you can see the random score being accumulated


# Training

Training the model to select action based on obs env

Before starting the training, we need to create a directory for logging
the updated weights

In [ ]:
#Create directory and make a variable of that directory
# Log path of the taraing
log_path = os.path.join("Training", "Logs")

In [34]:
log_path

'Training\\Logs'

Again recreating the env variable,
Making a dummy vec env for vectorizing a single env
and setting up the model with PPO, env and saving the logs in log path

In [ ]:
env=gym.make(env_name, render_mode="human") # redefining the environment
env = DummyVecEnv([lambda: env])  # Wrap the environment in a DummyVecEnv for vectorized training
# Create the PPO model with the specified policy and environment
model= PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device


Begin learning using the below command and set a total timesteps to terminate

In [36]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 1056 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 906         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008801829 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.0057      |
|    learning_rate        | 0.0003      |
|    loss                 | 6.2         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0165     |
|    value_loss           | 52.5        |
-----------------------------------------
---

Next task is to save the model,

In [ ]:
# Firstly create the directory and variable of the path to the directory
# Save the PPO model
PPO_Path = os.path.join("Training", "Saved Models", "PPO_CartPole")

In [ ]:
# Save the model using this command
model.save(PPO_Path)

Overall Consolidated Training a model Looks like this

In [ ]:
log_path = os.path.join("Training", "Logs")
PPO_Path = os.path.join("Training", "Saved Models", "PPO_CartPole")
env=gym.make(env_name, render_mode="human") # redefining the environment
env = DummyVecEnv([lambda: env])  # Wrap the environment in a DummyVecEnv for vectorized training
model= PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)# Create the PPO model with the specified policy and environment
model.learn(total_timesteps=20000)
model.save(PPO_Path)


Now you can delete the variable model so that I can demonstrate howe to load model

In [ ]:
del model  # Delete the model to demonstrate loading

In [5]:
# Load the model using the same path as above
# Load the PPO model
model = PPO.load(PPO_Path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# Evaluation

Once the model is loaded now you can evaluate the policy using another method as shown below

This evaluation is done based on given num random episodes and thier scores are averaged out 

In [ ]:
# Evaluate policy takes arguments as , model, env. no. of episodes to evaluate, and render mode
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(500.0, 0.0)

# Testing

While Evalute policy doesnot give you enough insites of each episodes, we create Testing 

In [ ]:
# Now all the code is same as above but we will use
# trained model to predict the action and then take a step in the environment
episodes = 10
for episode in range(1, episodes + 1):
    obs ,info= env.reset()
    done = False
    score=0

    while not done:
        # olnly change this line to use the trained model to predict the action
        action,_= model.predict(obs)
        # action= env.action_space.sample()  # Random action for demonstration
        obs, reward, done, info,_ = env.step(action)
        env.render()
        score += reward
        # print(f"Episode: {episode}, Score: {score}, Action: {action}, Reward: {reward}")
    print(f"Episode {episode} finished with score: {score}")
env.close()
    

Episode 1 finished with score: 279.0
Episode 2 finished with score: 1615.0
Episode 3 finished with score: 759.0
Episode 4 finished with score: 729.0
Episode 5 finished with score: 96.0
Episode 6 finished with score: 419.0
Episode 7 finished with score: 828.0
Episode 8 finished with score: 176.0
Episode 9 finished with score: 86.0
Episode 10 finished with score: 188.0
